In [4]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import cv2
import os
import sys
sys.path.append("MODNet/src")


from models.modnet import MODNet

class HairSegmenter:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        
        self.device = device
        
        # Инициализация архитектуры
        self.model = MODNet(backbone_pretrained=False)
        self.model = nn.DataParallel(self.model)
        self.model = self.model.to(self.device)
        
        # Загрузка весов
        ckpt_path = "MODNet/pretrained/modnet_photographic_portrait_matting.ckpt"
        weights = torch.load(ckpt_path, map_location=self.device)
        self.model.load_state_dict(weights)
        self.model.eval()

    def preprocess_image(self, image_path):
        
        image = Image.open(image_path).convert('RGB')
        self.orig_size = image.size
        
        im = np.asarray(image).astype(np.float32)
        im = cv2.resize(im, (512, 512))
        
        im = im / 255.0
        im = (im - 0.5) / 0.5
        
        im = torch.from_numpy(im).permute(2, 0, 1).unsqueeze(0)
        
        return im.to(self.device)

    def predict(self, image_path, output_path='results/output.png'):
        
        input_tensor = self.preprocess_image(image_path)
        
        with torch.no_grad():
            _, _, matte = self.model(input_tensor, True)
        
        matte = matte[0][0].data.cpu().numpy()
        matte = (matte * 255).astype(np.uint8)
        
        matte_pil = Image.fromarray(matte)
        matte_pil = matte_pil.resize(self.orig_size, Image.BILINEAR)
        
        original = Image.open(image_path).convert('RGBA')
        original.putalpha(matte_pil)
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        original.save(output_path)
        
        print(f"Result saved to {output_path}")
        
if __name__ == "__main__":
    segmenter = HairSegmenter()
    if os.path.exists('data/input.jpeg'):
        segmenter.predict('data/input.jpeg', 'results/portrait_matte.png')
    else:
        print("Please put an image at data/input.jpg")


Result saved to results/portrait_matte.png


In [6]:
import cv2
import numpy as np
from PIL import Image
import os

def refine_hair_mask(portrait_image_path, matte_image_path, output_path='results/hair_only.png'):

    
    # === ИСПРАВЛЕНИЕ: Загрузка через PIL (работает с кириллицей) ===
    try:
        original_pil = Image.open(portrait_image_path).convert('RGB')
        matte_pil = Image.open(matte_image_path).convert('RGBA')
    except Exception as e:
        raise FileNotFoundError(f"Failed to load images: {e}")
    
    # Конвертация PIL -> OpenCV (BGR)
    original = cv2.cvtColor(np.array(original_pil), cv2.COLOR_RGB2BGR)
    
    # Для матты берем альфа-канал
    matte_rgba = np.array(matte_pil)
    if matte_rgba.shape[2] == 4:
        matte = matte_rgba[:, :, 3]  # Альфа-канал
    else:
        matte = cv2.cvtColor(matte_rgba, cv2.COLOR_RGB2GRAY)

    # === 1. Бинарная маска ===
    _, binary_mask = cv2.threshold(matte, 25, 255, cv2.THRESH_BINARY)
    
    # === 2. Контур головы ===
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        h, w = matte.shape
        x, y, w, h = int(w*0.25), int(h*0.1), int(w*0.5), int(h*0.6)
    else:
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
    
    # === 3. Зона волос (верхняя часть) ===
    hair_roi_y = int(y) 
    hair_roi_h = int(h * 0.45)
    
    hair_zone_mask = np.zeros_like(matte)
    cv2.rectangle(hair_zone_mask, (x, hair_roi_y), (x + w, hair_roi_y + hair_roi_h), 255, -1)
    
    # === 4. Объединение масок ===
    hair_mask = cv2.bitwise_and(matte, matte, mask=hair_zone_mask)
    
    # === 5. Морфология ===
    kernel = np.ones((5,5), np.uint8)
    dilated_mask = cv2.dilate(hair_mask, kernel, iterations=2)
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)
    
    # === 6. Сохранение результата ===
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Создаем RGBA изображение
    b, g, r = cv2.split(original)
    rgba = [b, g, r, eroded_mask]
    dst = cv2.merge(rgba, 4)
    
    cv2.imwrite(output_path, dst)
    print(f"✅ Hair mask saved to {output_path}")
    
    # === Отладочные файлы ===
    cv2.imwrite('results/debug_binary_mask.png', binary_mask)
    cv2.imwrite('results/debug_hair_zone.png', hair_zone_mask)
    cv2.imwrite('results/debug_final_mask.png', eroded_mask)

if __name__ == "__main__":
    refine_hair_mask('data/input.jpeg', 'results/portrait_matte.png')

✅ Hair mask saved to results/hair_only.png


In [7]:
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image
import os

class FaceLandmarkDetector:
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=False,
            min_detection_confidence=0.5
        )
        
        # Ключевые точки для волос (MediaPipe Face Mesh)
        self.hairline_points = [
            10, 151, 162, 163, 164, 165, 166, 167,  # Линия волос спереди
            12, 13, 14, 15, 16, 17,  # Лоб
        ]
        
    def get_hair_region(self, image_path):
        """
        Возвращает bounding box зоны волос на основе лэндмарков.
        Исправлено: загрузка через PIL для поддержки кириллических путей.
        """
        # === ИСПРАВЛЕНИЕ: Загрузка через PIL ===
        try:
            image_pil = Image.open(image_path).convert('RGB')
            image_np = np.array(image_pil)
            image_rgb = cv2.cvtColor(image_np, cv2.COLOR_RGB2RGB)  # PIL уже RGB
        except Exception as e:
            print(f"⚠️  Failed to load image: {e}")
            return None
        
        height, width = image_rgb.shape[:2]
        
        results = self.face_mesh.process(image_rgb)
        
        if not results.multi_face_landmarks:
            print("⚠️  No face detected! Using fallback heuristic.")
            return None
        
        landmarks = results.multi_face_landmarks[0].landmark
        
        # Находим верхнюю точку лба/линии волос
        top_y = height
        left_x = width
        right_x = 0
        
        for idx in self.hairline_points:
            if idx < len(landmarks):
                point = landmarks[idx]
                px = int(point.x * width)
                py = int(point.y * height)
                
                top_y = min(top_y, py)
                left_x = min(left_x, px)
                right_x = max(right_x, px)
        
        # Добавляем padding вверх для захвата прически
        padding_top = int(height * 0.15)
        padding_sides = int(width * 0.1)
        
        hair_bbox = {
            'x': max(0, left_x - padding_sides),
            'y': max(0, top_y - padding_top),
            'w': min(width, right_x + padding_sides) - max(0, left_x - padding_sides),
            'h': max(0, top_y - padding_top) + int(height * 0.25)
        }
        
        print(f"📍 Hair region from landmarks: {hair_bbox}")
        return hair_bbox
    
    def close(self):
        self.face_mesh.close()


def refine_hair_mask_with_landmarks(portrait_image_path, matte_image_path, output_path='results/hair_only.png'):
    """
    Улучшенная версия с использованием MediaPipe Face Mesh.
    """
    # === Загрузка изображений (через PIL для кириллицы) ===
    original_pil = Image.open(portrait_image_path).convert('RGB')
    matte_pil = Image.open(matte_image_path).convert('RGBA')
    
    original = cv2.cvtColor(np.array(original_pil), cv2.COLOR_RGB2BGR)
    matte_rgba = np.array(matte_pil)
    
    if matte_rgba.shape[2] == 4:
        matte = matte_rgba[:, :, 3]
    else:
        matte = cv2.cvtColor(matte_rgba, cv2.COLOR_RGB2GRAY)
    
    height, width = matte.shape
    
    # === Получаем зону волос из лэндмарков ===
    detector = FaceLandmarkDetector()
    hair_bbox = detector.get_hair_region(portrait_image_path)
    detector.close()
    
    # === Fallback если лицо не найдено ===
    if hair_bbox is None:
        contours, _ = cv2.findContours(
            cv2.threshold(matte, 25, 255, cv2.THRESH_BINARY)[1], 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        )
        if contours:
            c = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            hair_bbox = {
                'x': x,
                'y': y,
                'w': w,
                'h': int(h * 0.25)
            }
    
    # === Создаем маску зоны волос ===
    hair_zone_mask = np.zeros_like(matte)
    cv2.rectangle(
        hair_zone_mask,
        (hair_bbox['x'], hair_bbox['y']),
        (hair_bbox['x'] + hair_bbox['w'], hair_bbox['y'] + hair_bbox['h']),
        255,
        -1
    )
    
    # === Объединение с маттой ===
    hair_mask = cv2.bitwise_and(matte, matte, mask=hair_zone_mask)
    
    # === Морфология ===
    kernel = np.ones((5, 5), np.uint8)
    dilated_mask = cv2.dilate(hair_mask, kernel, iterations=2)
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)
    
    # === Сохранение ===
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    b, g, r = cv2.split(original)
    rgba = [b, g, r, eroded_mask]
    dst = cv2.merge(rgba, 4)
    
    cv2.imwrite(output_path, dst)
    cv2.imwrite('results/debug_hair_zone_landmarks.png', hair_zone_mask)
    cv2.imwrite('results/debug_final_mask_landmarks.png', eroded_mask)
    
    print(f"✅ Hair mask saved to {output_path}")


if __name__ == "__main__":
    refine_hair_mask_with_landmarks('data/input.jpeg', 'results/portrait_matte.png')

⚠️  Failed to load image: module 'cv2' has no attribute 'COLOR_RGB2RGB'
✅ Hair mask saved to results/hair_only.png
